Code Author: Tanzid Sultan

**Convolutional Neural Network**:
We will now replace the hidden layer from our simple neural network model with a `convolutional layer`. In the usual hidden layer, we multiply the input vector `L0` with a weights matrix `W0`. When the input size is large, i.e. each instance has a large number of feature attributes (e.g. in the case of image data with lots of pixels), we end up with a large number of weights in `W0`. This can lead to the model overfitting the training data and lower the accuracy of the predictions. This problem can be mitigated by introducing a smaller weights matrix, also called a `kernel`, and applying this `kernel` repeatedly over different subsections of the data. So for example, if we have a 28x28 (=784) pixel image input, then instead of multiplying with a weight matrix with 28x28 rows, we can use a 6x6 kernel and multiply it with every 6x6 subsection of the image. We can also use multiple different kernels to process the inputs and pass on a combination of the different kernel outputs onto the next layer.         

In [10]:
import numpy as np

'''
    Input layer class: Input layer does not perform any operations
'''
class input_layer(object):
    '''
        class constructor
    '''
    def __init__(self) -> None:
        pass

    ''' 
        Input layer forward pass
    '''
    def forward(self, L_0):
        self.L_0 = L_0
        return self.L_0
    
''' 
    Hidden layer class: Hidden layer performs 2 operations. First it performs matrix multiplication
                        of inputs L_0 with weights W_0. Then it operates on this result with the activation
                        function.
'''    
class hidden_layer(object):
    '''
        class constructor
    '''
    def __init__(self, W, activation) -> None:
        self.W = W
        self.W_grad = np.zeros_like(W)
        self.activation = activation
        print(f"Hidden layer activation function: {activation}")

    ''' 
        Hidden layer forward propagation
    '''
    def forward(self, L, dropout): 
        self.L = L
        return self.forward_matrix_mult(dropout)

    def forward_matrix_mult(self, dropout):
        self.Z =  np.dot(self.L, self.W)
        self.dropout = dropout 
        if(self.dropout):
            # generate a random dropout mask with rougly equal numbers of 0s and 1s
            self.dropout_mask = np.random.randint(0,2,size=(self.Z.shape))
           
        if(self.activation == "relu"):
            if(self.dropout):
              # multiply by a factor of 2 to compensate for rougly 1/2 the neurons being turned off by the masking
              return 2 * self.dropout_mask * self.forward_relu()
            else:
                return self.forward_relu()
    
        elif(self.activation == "sigmoid"):
            if(self.dropout):
              return 2 * self.dropout_mask * self.forward_sigmoid()
            else:
                return self.forward_sigmoid()
    
        elif(self.activation == "tanh"):
            if(self.dropout):
              return 2 * self.dropout_mask * self.forward_tanh()
            else:
                return self.forward_tanh()
    
    def forward_relu(self):
        return Relu(self.Z)
    
    def forward_sigmoid(self):
        return sigmoid(self.Z)
   
    def forward_tanh(self):
        return tanh(self.Z)
    
    ''' 
        Hidden layer backpropagation of derivatives
    '''
    def backward(self, D):
        if(self.activation == "relu"):
           self.backward_relu(D)
        elif(self.activation == "sigmoid"):
           self.backward_sigmoid(D)
        elif(self.activation == "tanh"):
           self.backward_tanh(D)

    def backward_relu(self, D):
        # dE/dZ
        dE_dZ = D * Relu_deriv(self.Z) 
        self.backward_matrix_mult(dE_dZ)
    
    def backward_sigmoid(self, D):
        # dE/dZ
        dE_dZ = D * sigmoid_deriv(self.Z) 
        self.backward_matrix_mult(dE_dZ)
    
    def backward_tanh(self, D):
        # dE/dZ
        dE_dZ = D * tanh_deriv(self.Z) 
        self.backward_matrix_mult(dE_dZ)
    
    def backward_matrix_mult(self, D):
        # dE/dW0
        if(self.dropout):
            self.W_grad = np.dot((self.L).T, self.dropout_mask * D)
        else:
            self.W_grad = np.dot((self.L).T, D)

    ''' 
        Gradient descent optimization of hidden layer weights
    '''
    def update_weights(self, alpha):
        self.W -= alpha * self.W_grad

''' 
    Convolutional layer class: This layer performs two operations. First, it computes the matrix multiplication of subsections odf the input with the kernels. Then it operates on the result with the activation function
'''
class convolutional_layer(object):

    '''
        class constructor
    '''
    def __init__(self, K, image_rows, image_cols, kernel_rows, kernel_cols, activation) -> None:
        self.K = K
        self.image_cols = image_cols
        self.image_rows = image_rows
        self.kernel_cols = kernel_cols
        self.kernel_rows = kernel_rows
        self.activation = activation
        self.K_grad = np.zeros_like(K)

    
    ''' 
        convolutional layer forward propagation
    '''
    def forward(self, L, dropout):
       
        # reshape the input image array
        L = L.reshape(L.shape[0], self.image_rows, self.image_cols)

        # get all sub-sections from the image
        sections = []
        for i in range(self.image_rows-self.kernel_rows+1):
            for j in range(self.image_cols-self.kernel_cols+1):
                section = L[:,i:i+self.kernel_rows, j:j+self.kernel_cols]   
                section = section.reshape(-1,1,self.kernel_rows,self.kernel_cols)
                sections.append(section)
     
        # concatenate all sections into a single array
        expanded_input = np.concatenate(sections, axis=1)    
        input_shape = expanded_input.shape 
        
        # flatten the sections
        self.flattened_input = expanded_input.reshape(expanded_input.shape[0]*expanded_input.shape[1], -1) 

        return self.kernel_mult(input_shape, dropout)
    
    def kernel_mult(self, input_shape, dropout):

        # matrix multiplication of flattened image sections with kernels
        self.Z = np.dot(self.flattened_input, self.K) 
        
        # flatten the kernel output for each image
        self.Zflat = self.Z.copy()        
        self.Zflat = self.Zflat.reshape(input_shape[0], -1)        
    
        self.dropout = dropout 
        if(self.dropout):
            # generate a random dropout mask with rougly equal numbers of 0s and 1s
            self.dropout_mask = np.random.randint(0,2,size=(self.Zflat.shape))
    
        if(self.activation == "relu"):
            if(self.dropout):
              # multiply by a factor of 2 to compensate for rougly 1/2 the neurons being turned off by the masking
              return 2 * self.dropout_mask * self.forward_relu()
            else:
                return self.forward_relu()
    
        elif(self.activation == "sigmoid"):
            if(self.dropout):
              return 2 * self.dropout_mask * self.forward_sigmoid()
            else:
                return self.forward_sigmoid()
    
        elif(self.activation == "tanh"):
            if(self.dropout):
              return 2 * self.dropout_mask * self.forward_tanh()
            else:
                return self.forward_tanh()
    
    def forward_relu(self):
        return Relu(self.Zflat)
    
    def forward_sigmoid(self):
        return sigmoid(self.Zflat)
   
    def forward_tanh(self):
        return tanh(self.Zflat)
    
    ''' 
        convolutional layer backpropagation
    '''
    def backward(self, D):
        if(self.activation == "relu"):
           self.backward_relu(D)
        elif(self.activation == "sigmoid"):
           self.backward_sigmoid(D)
        elif(self.activation == "tanh"):
           self.backward_tanh(D)

    def backward_relu(self, D):
        # dE/dZ
        dE_dZ = D * Relu_deriv(self.Zflat) 
        self.backward_kernel_mult(dE_dZ)
    
    def backward_sigmoid(self, D):
        # dE/dZ
        dE_dZ = D * sigmoid_deriv(self.Zflat) 
        self.backward_kernel_mult(dE_dZ)
    
    def backward_tanh(self, D):
        # dE/dZ
        dE_dZ = D * tanh_deriv(self.Zflat) 
        self.backward_kernel_mult(dE_dZ)
    
    def backward_kernel_mult(self, D):
        # dE/dK
        if(self.dropout):
            self.Kgrad = np.dot((self.flattened_input).T, (self.dropout_mask * D).reshape(self.Z.shape))
        else:
            self.K_grad = np.dot((self.flattened_input).T, D.reshape(self.Z.shape))

    ''' 
        Gradient descent optimization of convolutional layer kernels
    '''
    def update_kernels(self, alpha):
        self.K -= alpha * self.K_grad


''' 
    Ouput layer class: Performs two operations, first matrix multiplication of inputs L_1 with weights
                       W_1. This result is then operated on by squared error function.  
'''
class output_layer(object):
    
    ''' 
        class constructor
    '''

    def __init__(self, W) -> None:
        self.W = W
        self.W_grad = np.zeros_like(W)

    ''' 
        Output layer forward propagation
    '''
    def forward(self, L, Y, soft):
        self.L = L
        self.Y = Y
        return self.forward_matrix_mult(soft)

    def forward_matrix_mult(self, soft):
        self.P = np.dot(self.L, self.W) 
        if(soft):
            self.P = softmax(self.P)
        
        return self.P, self.forward_error()
 
    def forward_error(self):
        return np.sum((self.P - self.Y)**2) / self.P.shape[0]

    '''     
        Output layer backpropagation of derivatives
    '''
    def backward(self):
        return self.backward_error()

    def backward_error(self):
        # dE/dP
        dE_dP = 2*(self.P - self.Y) / self.P.shape[0]
        return self.backward_matrix_mult(dE_dP)

    def backward_matrix_mult(self, D):
        # dE/dW1
        self.W_grad = np.dot((self.L).T, D)
        # dE/dL1
        dE_dL = np.dot(D, (self.W).T)
        return dE_dL
    
    ''' 
        Gradient descent optimization of output layer weights
    '''
    def update_weights(self, alpha):
        self.W -= alpha * self.W_grad

'''
    A 3-layer convolutional neural network class
'''
class three_layer_convolutional_net(object):
    ''' 
        class constructor: Takes in the following parameters- number of neurons in input layer (which is the number of feature attributes for each instance), number of hidden layers (has to be at least 1 and can be arbitrarily large), number of neurons in the output layer (which is the number of target attributes) and gradient descent step-size (alpha)
    '''
    def __init__(self, input_neurons, output_neurons, image_rows, image_cols, kernel_rows, kernel_cols, num_kernels, convolutional_layer_activation = "relu") -> None:
        self.input_neurons  = input_neurons
        self.output_neurons = output_neurons
        self.image_rows = image_rows
        self.image_cols = image_cols
        self.kernel_rows = kernel_rows
        self.kernel_cols = kernel_cols
        self.num_kernels = num_kernels

        np.random.seed(1)
        # initialize kernels for convolutional layer 
        K = 0.02*np.random.random(size=(self.kernel_rows*self.kernel_cols, self.num_kernels)) - 0.01

        hidden_neurons = (self.image_rows-self.kernel_rows+1) * (self.image_cols-self.kernel_cols+1) * self.num_kernels

        # initialize weights W1 between hidden layer and output layer
        W1 = 0.2*np.random.random(size=(hidden_neurons, output_neurons)) - 0.1 

        # initialize layer objects
        self.layer_0 = input_layer()
        self.layer_1 = convolutional_layer(K, self.image_rows, self.image_cols, self.kernel_rows, self.kernel_cols, activation=convolutional_layer_activation)
        self.layer_2 = output_layer(W1)

    ''' 
        neural network forward pass
    '''
    def forward_net(self, L0, Y, dropout, soft):
        # input layer forward pass
        self.L0 = self.layer_0.forward(L0) 
        # hidden layer forward pass 
        self.L1 = self.layer_1.forward(self.L0, dropout) 
        # output layer forward pass
        self.L2, error = self.layer_2.forward(self.L1, Y, soft) 

        return self.L2, error

    ''' 
        neural network backward pass
    ''' 
    def backward_net(self):
       # output layer backpropagation
       D = self.layer_2.backward() 
       # hidden layer backpropagation
       self.layer_1.backward(D) 

    '''     
        kernel and weight optimization
    '''
    def optimize(self, alpha):
        # update output layer weights
        self.layer_2.update_weights(alpha)
        # update hidden layer weights
        self.layer_1.update_kernels(alpha)

    '''     
        train the network
    ''' 
    def train(self, X_train, y_train, X_test, y_test, alpha, batch_size=1, niters=1, dropout=False, soft=False):
        print(f"Dropout Enabled: {dropout}")
        print(f"Softmax Enabled: {soft}")
        print(f"Batch size: {batch_size}")
        print(f"Alpha: {alpha}")
        print("Training in progress...")
        #training iterations
        for i in range(niters):
            total_error = 0.0
            train_correct_count = 0
            # train using batch of instances
            for j in range(int(X_train.shape[0]/batch_size)):

                lo = j * batch_size
                hi = min((j+1) * batch_size, X_train.shape[0])

                X = X_train[lo:hi]
                y = y_train[lo:hi]

                # forward propagation
                prediction, error = self.forward_net(X, y, dropout, soft)
                total_error += error
                
                for k in range(hi-lo):
                    train_correct_count += int(np.argmax(prediction[k]) == np.argmax(y[k]))
                
                #if(i == (niters-1)):
                #    print(f"Instance# {j+1}, Target: {y}, Prediction: {prediction}")

                # backpropagation
                self.backward_net()
                
                # weight optimization
                self.optimize(alpha)

            # predict using test instances
            test_correct_count = 0
            for j in range(X_test.shape[0]):
                X = X_test[j:j+1]
                y = y_test[j:j+1]

                # forward propagation
                prediction, error = self.forward_net(X, y, dropout=False, soft=False)
                test_correct_count += int(np.argmax(prediction) == np.argmax(y))

            print(f"Iteration# {i+1}, Total error: {total_error}, Training accuracy: {train_correct_count/len(y_train)}, Testing accuracy: {test_correct_count/len(y_test)}")

# Relu function
def Relu(x):
    return x*(x > 0)

# Relu derivative function
def Relu_deriv(x):
    return (x > 0)

def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    return (1.0 - np.tanh(x)**2)

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-1.0 * x))

def sigmoid_deriv(x):
    return sigmoid(x) * (1.0 - sigmoid(x)) 

def softmax(x): 
    ex = np.exp(x)
    return ex/np.sum(ex, axis = 1, keepdims = True)  



Testing our 3 layer convolutional neural network model with the `MNIST dataset` of handwritten digits

In [2]:
from keras.datasets import mnist

In [11]:
'''
    MNIST dataset of handwritten digits:

    Each observation is an image. The input values per image are 28x28 pixels (i.e. 784 features/inputs per observation).
'''

(X_train, y_train), (X_test, y_test) = mnist.load_data() # x values contain image pixels (i.e. features) and y vaues are the corresponding labels

print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")
print(f"X_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")

img_rows = X_train.shape[1]
img_cols = X_train.shape[2]

# flatten image pixels array
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]) 
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]) 

# normalize of pixel values from (0,255) to (0,1)
X_train = X_train / 255
X_test = X_test / 255

# one-hot encode the labels
y_train_onehot = np.zeros(shape=(y_train.shape[0], 10))
y_test_onehot = np.zeros(shape=(y_test.shape[0], 10))

for i in range(y_train.shape[0]):
    y_train_onehot[i, y_train[i]] = 1

for i in range(y_test.shape[0]):
    y_test_onehot[i, y_test[i]] = 1

# training dataset preparation
training_images = X_train[0:1000]
training_labels = y_train_onehot[0:1000]
testing_images = X_test
testing_labels = y_test_onehot

# initialize a three layer convolutional neural net object
three_net = three_layer_convolutional_net(input_neurons=training_images.shape[1], output_neurons=training_labels.shape[1], image_rows=img_rows, image_cols=img_cols, kernel_rows=3, kernel_cols=3, num_kernels=16, convolutional_layer_activation="tanh")

# train the net
three_net.train(training_images, training_labels, testing_images, testing_labels, alpha=0.05, batch_size=100, niters=300, dropout=True, soft=True)    

X_train shape = (60000, 28, 28)
y_train shape = (60000,)
X_test shape = (10000, 28, 28)
y_test shape = (10000,)
Dropout Enabled: True
Softmax Enabled: True
Batch size: 100
Alpha: 0.05
Training in progress...
Iteration# 1, Total error: 8.99563648783043, Training accuracy: 0.12, Testing accuracy: 0.1167
Iteration# 2, Total error: 8.982189246819486, Training accuracy: 0.144, Testing accuracy: 0.137
Iteration# 3, Total error: 8.973909173428853, Training accuracy: 0.155, Testing accuracy: 0.1606
Iteration# 4, Total error: 8.96136492440829, Training accuracy: 0.166, Testing accuracy: 0.1951
Iteration# 5, Total error: 8.94987377521972, Training accuracy: 0.21, Testing accuracy: 0.2339
Iteration# 6, Total error: 8.94157253184402, Training accuracy: 0.241, Testing accuracy: 0.2738
Iteration# 7, Total error: 8.931137270023754, Training accuracy: 0.262, Testing accuracy: 0.3142
Iteration# 8, Total error: 8.92264070092538, Training accuracy: 0.284, Testing accuracy: 0.3468
Iteration# 9, Total erro